# Load, Merge, and Clean Data

In [1]:
import pandas as pd
import os
import gc

# --- Configuration ---
BASE_DATA_PATH = '/Users/john/Downloads/osfstorage-archive'
EXPERIMENT_IDS_PATH = os.path.join(BASE_DATA_PATH, 'experiment_dataset_2021-09-23')

# --- 1. Get List of Experiment IDs (Subdirectories) ---
try:
    experiment_ids = [d for d in os.listdir(EXPERIMENT_IDS_PATH) if os.path.isdir(os.path.join(EXPERIMENT_IDS_PATH, d))]
    print(f"Found {len(experiment_ids)} experiment ID directories.")
except FileNotFoundError:
    print(f"Error: Directory not found at {EXPERIMENT_IDS_PATH}")
    experiment_ids = []

# --- 2. Generate File Paths ---
performance_file_paths = []
problems_file_paths = []
actions_file_paths = []
metrics_file_paths = []

for exp_id in experiment_ids:
    exp_path = os.path.join(EXPERIMENT_IDS_PATH, exp_id)
    performance_file_paths.append(os.path.join(exp_path, 'exp_alogs.csv'))
    problems_file_paths.append(os.path.join(exp_path, 'exp_plogs.csv'))
    actions_file_paths.append(os.path.join(exp_path, 'exp_slogs.csv'))
    metrics_file_paths.append(os.path.join(exp_path, 'priors.csv'))

# --- Define dtype Dictionaries ---
actions_dtypes = {
    'experiment_id': 'category',
    'student_id': 'category',
    'problem_id': 'category',
    'problem_part': 'category',
    'scaffold_id': 'category',
    'experiment_tag_path': 'category',
    'action': 'category',
    'assistments_reference_action_log_id': 'uint64'
}
actions_parse_dates = ['timestamp']

problems_dtypes = {
    'experiment_id': 'category',
    'student_id': 'category',
    'problem_id': 'category',
    'problem_part': 'category',
    'scaffold_id': 'category',
    'problem_condition': 'category',
    'session_count': 'uint16',
    'time_on_task': 'float32',
    'first_response_or_request_time': 'float32',
    'first_answer': 'string',
    'correct': 'boolean',
    'reported_score': 'float32',
    'answer_before_tutoring': 'boolean',
    'attempt_count': 'uint16',
    'hints_available': 'uint16',
    'hints_given': 'uint16',
    'scaffold_problems_available': 'uint16',
    'scaffold_problems_given': 'uint16',
    'explanation_available': 'boolean',
    'explanation_given': 'boolean',
    'answer_given': 'boolean',
    'assistments_reference_problem_log_id': 'uint64'
}
problems_parse_dates = ['start_time', 'end_time']

performance_dtypes = {
    'experiment_id': 'category',
    'student_id': 'category',
    'assignment_session_count': 'UInt16',
    'assigned_condition': 'category',
    'condition_time_on_task': 'float32',
    'condition_average_first_response_or_request_time': 'float32',
    'condition_problem_count': 'UInt16',
    'condition_total_correct': 'UInt16',
    'condition_total_correct_after_wrong_response': 'UInt16',
    'condition_total_correct_after_tutoring': 'UInt16',
    'condition_total_answers_before_tutoring': 'UInt16',
    'condition_total_attempt_count': 'UInt32',
    'condition_total_hints_available': 'UInt32',
    'condition_total_hints_given': 'UInt32',
    'condition_total_scaffold_problems_available': 'UInt32',
    'condition_total_scaffold_problems_given': 'UInt32',
    'condition_total_explanations_available': 'UInt32',
    'condition_total_explanations_given': 'UInt32',
    'condition_total_answers_given': 'UInt32',
    'condition_session_count': 'UInt16',
    'assistments_reference_assignment_log_id': 'uint64',
    'pretest_problem_count': 'UInt16',
    'pretest_correct': 'UInt16',
    'pretest_time_on_task': 'float32',
    'pretest_average_first_response_time': 'float32',
    'pretest_session_count': 'UInt16',
    'posttest_problem_count': 'UInt16',
    'posttest_correct': 'UInt16',
    'posttest_time_on_task': 'float32',
    'posttest_average_first_response_time': 'float32',
    'posttest_session_count': 'UInt16'
}
performance_parse_dates = ['release_date', 'due_date', 'start_time', 'end_time']

metrics_dtypes = {
    'experiment_id': 'category',
    'student_id': 'category',
    'student_prior_started_skill_builder_count': 'UInt32',
    'student_prior_completed_skill_builder_count': 'UInt32',
    'student_prior_started_problem_set_count': 'UInt32',
    'student_prior_completed_problem_set_count': 'UInt32',
    'student_prior_completed_problem_count': 'UInt32',
    'student_prior_median_first_response_time': 'float32',
    'student_prior_median_time_on_task': 'float32',
    'student_prior_average_correctness': 'float32',
    'student_prior_average_attempt_count': 'float32',
    'class_id': 'category',
    'class_student_count': 'uint16',
    'class_prior_skill_builder_count': 'UInt32',
    'class_prior_problem_set_count': 'UInt32',
    'class_prior_skill_builder_percent_started': 'float32',
    'class_prior_skill_builder_percent_completed': 'float32',
    'class_prior_problem_set_percent_started': 'float32',
    'class_prior_problem_set_percent_completed': 'float32',
    'class_prior_completed_problem_count': 'UInt32',
    'class_prior_median_time_on_task': 'float32',
    'class_prior_median_first_response_time': 'float32',
    'class_prior_average_correctness': 'float32',
    'class_prior_average_attempt_count': 'float32',
    'teacher id': 'category',
    'district_id': 'category',
    'location': 'category',
    'opportunity_zone': 'category',
    'locale_description': 'category'
}
metrics_parse_dates = ['class_creation_date', 'teacher_account_creation_date']


# --- Helper Function for Memory-Efficient Concatenation ---
def combine_csvs(file_paths, dtypes=None, parse_dates_list=None, usecols=None):
    """
    Reads multiple CSVs incrementally and concatenates them.
    (Function definition remains the same as previous response)
    """
    all_dfs = []
    print(f"\nProcessing {len(file_paths)} files...")
    for i, file_path in enumerate(file_paths):
        if i % 10 == 0:
             print(f"  Reading file {i+1}/{len(file_paths)}: {os.path.basename(os.path.dirname(file_path))}")
        try:
            df_temp = pd.read_csv(
                file_path,
                dtype=dtypes,
                parse_dates=parse_dates_list,
                usecols=usecols
            )
            all_dfs.append(df_temp)
            del df_temp
            if i % 10 == 9:
                 gc.collect()
        except FileNotFoundError:
            print(f"  Warning: File not found, skipping: {file_path}")
        except pd.errors.EmptyDataError:
            print(f"  Warning: File is empty, skipping: {file_path}")
        except ValueError as ve:
             print(f"  Error reading {file_path} (check dtypes/file content?): {ve}")
        except Exception as e:
            print(f"  Error reading {file_path}: {e}")

    if not all_dfs:
        print("  No dataframes were loaded.")
        return None

    print(f"Concatenating {len(all_dfs)} dataframes...")
    try:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print("Concatenation complete.")
    except Exception as e:
        print(f"Error during concatenation: {e}")
        combined_df = None

    del all_dfs
    gc.collect()
    return combined_df

# --- 3. Combine DataFrames for each type ---

print("Combining Performance Data (exp_alogs)...")
performance_df = combine_csvs(
    performance_file_paths,
    dtypes=performance_dtypes,
    parse_dates_list=performance_parse_dates
)
if performance_df is not None:
    print("\nPerformance DataFrame Info:")
    performance_df.info(memory_usage='deep')

Found 88 experiment ID directories.
Combining Performance Data (exp_alogs)...

Processing 88 files...
  Reading file 1/88: PSAU85Y
  Reading file 11/88: PSAYCFH
  Reading file 21/88: PSAZ2G4
  Reading file 31/88: PSAQJFP
  Reading file 41/88: PSAJVP8
  Reading file 51/88: PSA9XWV
  Reading file 61/88: PSAM4NK
  Reading file 71/88: PSATP2Z
  Reading file 81/88: PSASDZY
Concatenating 88 dataframes...
Concatenation complete.

Performance DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95990 entries, 0 to 95989
Data columns (total 35 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   experiment_id                                     95990 non-null  object 
 1   student_id                                        95990 non-null  object 
 2   release_date                                      95990 non-null  object 
 3   due_date                            

In [2]:
print("\nCombining Problems Data (exp_plogs)...")
problems_df = combine_csvs(problems_file_paths, dtypes=problems_dtypes)
if problems_df is not None:
    print("\nProblems DataFrame Info:")
    problems_df.info(memory_usage='deep')


Combining Problems Data (exp_plogs)...

Processing 88 files...
  Reading file 1/88: PSAU85Y
  Reading file 11/88: PSAYCFH
  Reading file 21/88: PSAZ2G4
  Reading file 31/88: PSAQJFP
  Reading file 41/88: PSAJVP8
  Reading file 51/88: PSA9XWV
  Reading file 61/88: PSAM4NK
  Reading file 71/88: PSATP2Z
  Reading file 81/88: PSASDZY
Concatenating 88 dataframes...
Concatenation complete.

Problems DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771386 entries, 0 to 771385
Data columns (total 24 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   experiment_id                         771386 non-null  object 
 1   student_id                            771386 non-null  object 
 2   problem_id                            771386 non-null  object 
 3   problem_part                          771386 non-null  object 
 4   scaffold_id                           52637 non-null   object 

In [3]:
print("\nCombining Actions Data (exp_slogs)...")
actions_df = combine_csvs(actions_file_paths, dtypes=actions_dtypes)
if actions_df is not None:
    print("\nActions DataFrame Info:")
    actions_df.info(memory_usage='deep')


Combining Actions Data (exp_slogs)...

Processing 88 files...
  Reading file 1/88: PSAU85Y
  Reading file 11/88: PSAYCFH
  Reading file 21/88: PSAZ2G4
  Reading file 31/88: PSAQJFP
  Reading file 41/88: PSAJVP8
  Reading file 51/88: PSA9XWV
  Reading file 61/88: PSAM4NK
  Reading file 71/88: PSATP2Z
  Reading file 81/88: PSASDZY
Concatenating 88 dataframes...
Concatenation complete.

Actions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3708299 entries, 0 to 3708298
Data columns (total 9 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   experiment_id                        object
 1   student_id                           object
 2   problem_id                           object
 3   problem_part                         object
 4   scaffold_id                          object
 5   experiment_tag_path                  object
 6   action                               object
 7   timestamp                     

In [4]:
print("\nCombining Metrics Data (priors)...")
metrics_df = combine_csvs(metrics_file_paths, dtypes=metrics_dtypes)
if metrics_df is not None:
    print("\nMetrics DataFrame Info:")
    metrics_df.info(memory_usage='deep')


Combining Metrics Data (priors)...

Processing 88 files...
  Reading file 1/88: PSAU85Y
  Reading file 11/88: PSAYCFH
  Reading file 21/88: PSAZ2G4
  Reading file 31/88: PSAQJFP
  Reading file 41/88: PSAJVP8
  Reading file 51/88: PSA9XWV
  Reading file 61/88: PSAM4NK
  Reading file 71/88: PSATP2Z
  Reading file 81/88: PSASDZY
Concatenating 88 dataframes...
Concatenation complete.

Metrics DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95979 entries, 0 to 95978
Data columns (total 31 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   experiment_id                                95979 non-null  object 
 1   student_id                                   95979 non-null  object 
 2   student_prior_started_skill_builder_count    95979 non-null  UInt32 
 3   student_prior_completed_skill_builder_count  95979 non-null  UInt32 
 4   student_prior_started_problem_set

In [5]:
# --- 4. Merge DataFrames into One ---

merged_df = None
merge_successful = True

print("\n--- Starting Merge Operations ---")

# Check if base DataFrame (actions_df) exists
if actions_df is not None and not actions_df.empty:
    # Use actions_df as the base because it's likely the most granular
    merged_df = actions_df.copy() # Start with a copy
    print(f"Starting with actions_df: {merged_df.shape}")

    # Merge Problems Data
    if problems_df is not None and not problems_df.empty:
        try:
            print("Merging problems_df...")
            # Define keys common to actions and problems
            problem_keys = ['experiment_id', 'student_id', 'problem_id', 'problem_part', 'scaffold_id']
            # Check if all keys exist in both DataFrames before merging
            keys_exist_problems = all(key in problems_df.columns for key in problem_keys)
            keys_exist_merged = all(key in merged_df.columns for key in problem_keys)

            if keys_exist_problems and keys_exist_merged:
                 # Use suffixes to distinguish overlapping columns not used in merge keys
                merged_df = pd.merge(
                    merged_df,
                    problems_df,
                    on=problem_keys,
                    how='left',
                    suffixes=('_action', '_problem') # Keep original actions cols, add _problem suffix to new ones
                 )
                print(f"After merging problems_df: {merged_df.shape}")
                del problems_df # Free memory
                gc.collect()
            else:
                 print("Warning: Cannot merge problems_df - required keys missing.")
                 print(f"  Keys in merged: {[k for k in problem_keys if k in merged_df.columns]}")
                 print(f"  Keys in problems_df: {[k for k in problem_keys if k in problems_df.columns]}")
                 merge_successful = False
        except Exception as e:
            print(f"Error merging problems_df: {e}")
            merge_successful = False
    else:
        print("Skipping problems_df merge (not loaded or empty).")

    # Merge Performance Data
    if performance_df is not None and not performance_df.empty and merge_successful:
        try:
            print("Merging performance_df...")
            perf_keys = ['experiment_id', 'student_id']
            # Check keys exist
            keys_exist_perf = all(key in performance_df.columns for key in perf_keys)
            keys_exist_merged = all(key in merged_df.columns for key in perf_keys)

            if keys_exist_perf and keys_exist_merged:
                merged_df = pd.merge(
                    merged_df,
                    performance_df,
                    on=perf_keys,
                    how='left',
                    suffixes=('', '_perf') # Keep existing cols, add _perf to new overlapping ones
                )
                print(f"After merging performance_df: {merged_df.shape}")
                del performance_df # Free memory
                gc.collect()
            else:
                 print("Warning: Cannot merge performance_df - required keys missing.")
                 print(f"  Keys in merged: {[k for k in perf_keys if k in merged_df.columns]}")
                 print(f"  Keys in performance_df: {[k for k in perf_keys if k in performance_df.columns]}")
                 merge_successful = False
        except Exception as e:
            print(f"Error merging performance_df: {e}")
            merge_successful = False
    else:
         print("Skipping performance_df merge (not loaded, empty, or previous merge failed).")


    # Merge Metrics Data
    if metrics_df is not None and not metrics_df.empty and merge_successful:
        try:
            print("Merging metrics_df...")
            metrics_keys = ['experiment_id', 'student_id']
            # Check keys exist
            keys_exist_metrics = all(key in metrics_df.columns for key in metrics_keys)
            keys_exist_merged = all(key in merged_df.columns for key in metrics_keys)

            if keys_exist_metrics and keys_exist_merged:
                merged_df = pd.merge(
                    merged_df,
                    metrics_df,
                    on=metrics_keys,
                    how='left',
                    suffixes=('', '_metrics') # Keep existing cols, add _metrics to new overlapping ones
                )
                print(f"After merging metrics_df: {merged_df.shape}")
                del metrics_df # Free memory
                gc.collect()
            else:
                print("Warning: Cannot merge metrics_df - required keys missing.")
                print(f"  Keys in merged: {[k for k in metrics_keys if k in merged_df.columns]}")
                print(f"  Keys in metrics_df: {[k for k in metrics_keys if k in metrics_df.columns]}")
                merge_successful = False
        except Exception as e:
            print(f"Error merging metrics_df: {e}")
            merge_successful = False
    else:
        print("Skipping metrics_df merge (not loaded, empty, or previous merge failed).")

    # Final check and info
    if merged_df is not None and merge_successful:
        print("\n--- Merge Complete ---")
        print("Final Merged DataFrame Info:")
        merged_df.info(memory_usage='deep')
        print(merged_df.head())
        # Clean up base df if needed
        del actions_df
        gc.collect()
    elif merged_df is not None:
         print("\n--- Merge Partially Complete (some steps failed/skipped) ---")
         print("Partial Merged DataFrame Info:")
         merged_df.info(memory_usage='deep')
         print(merged_df.head())
         # Clean up base df if needed
         del actions_df
         gc.collect()
    else:
        print("\n--- Merge Failed or Base DataFrame Empty ---")

else:
    print("\n--- Merge Skipped: Base DataFrame (actions_df) not loaded or empty ---")


--- Starting Merge Operations ---
Starting with actions_df: (3708299, 9)
Merging problems_df...
After merging problems_df: (3708299, 28)
Merging performance_df...
After merging performance_df: (3711215, 61)
Merging metrics_df...
After merging metrics_df: (3711215, 90)

--- Merge Complete ---
Final Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711215 entries, 0 to 3711214
Data columns (total 90 columns):
 #   Column                                            Dtype  
---  ------                                            -----  
 0   experiment_id                                     object 
 1   student_id                                        object 
 2   problem_id                                        object 
 3   problem_part                                      object 
 4   scaffold_id                                       object 
 5   experiment_tag_path                               object 
 6   action                                            objec

# Data Cleaning

In [6]:
import numpy as np

# --- Step 1: Initial Inspection ---
# Get a comprehensive overview of the merged data
print("--- Initial DataFrame Info ---")

if 'merged_df' in locals() and merged_df is not None:
    merged_df.info(verbose=True, show_counts=True, memory_usage='deep')

    print("\n--- First 5 Rows ---")
    print(merged_df.head())

    print("\n--- List of All Columns ---")
    # This helps identify columns with suffixes like _action, _problem, _perf, _metrics
    all_columns = merged_df.columns.tolist()
    print(all_columns)

    # Check shape
    print(f"\nShape: {merged_df.shape}")
else:
    print("Error: merged_df not found or is None. Please ensure Step 4 ran correctly.")

--- Initial DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711215 entries, 0 to 3711214
Data columns (total 90 columns):
 #   Column                                            Non-Null Count    Dtype  
---  ------                                            --------------    -----  
 0   experiment_id                                     3711215 non-null  object 
 1   student_id                                        3711215 non-null  object 
 2   problem_id                                        2880476 non-null  object 
 3   problem_part                                      2880476 non-null  object 
 4   scaffold_id                                       197410 non-null   object 
 5   experiment_tag_path                               3489249 non-null  object 
 6   action                                            3711215 non-null  object 
 7   timestamp                                         3711215 non-null  object 
 8   assistments_reference_action_log_id      

In [7]:
# --- Step 2: Handle Duplicate/Suffixed Columns & Correct Initial Dtypes ---

if 'merged_df' in locals() and merged_df is not None:
    print("\n--- Step 2: Renaming Columns & Correcting Dtypes ---")

    # 2a. Renaming based on observations
    rename_dict = {
        'start_time_perf': 'assignment_start_time', # From performance_df
        'end_time_perf': 'assignment_end_time',   # From performance_df
        'assistments_reference_action_log_id': 'action_log_id', # Make shorter
        # 'assistments_reference_problem_log_id': 'problem_log_id', # This column is empty, rename if needed later
        'assistments_reference_assignment_log_id': 'assignment_log_id' # Make shorter
        # NOTE: The original 'start_time', 'end_time' from problems_df are empty,
        # so we won't rename them for now. They might be dropped later.
    }
    # Keep only renames for columns that actually exist
    rename_dict = {k: v for k, v in rename_dict.items() if k in merged_df.columns}

    if rename_dict:
        print("Applying renames:", rename_dict)
        merged_df.rename(columns=rename_dict, inplace=True)
    else:
        print("No columns identified for renaming in this example.")

    # 2b. Correcting Data Types for 'object' columns

    # Columns to convert to 'category' (IDs and low-cardinality text)
    cols_to_category = [
        'experiment_id', 'student_id', 'problem_id', 'problem_part', 'scaffold_id',
        'experiment_tag_path', 'action', 'assigned_condition', 'class_id',
        'district_id', 'location', 'opportunity_zone', 'locale_description',
    ]
    # Filter list to only include columns actually present and are object type
    cols_to_category = [col for col in cols_to_category if col in merged_df.columns and merged_df[col].dtype == 'object']

    if cols_to_category:
        print(f"\nConverting {len(cols_to_category)} object columns to category...")
        for col in cols_to_category:
            try:
                merged_df[col] = merged_df[col].astype('category')
            except Exception as e:
                print(f"  Error converting {col} to category: {e}")
        print("Category conversion attempted.")
    else:
        print("\nNo object columns identified for category conversion in the list.")

    # Also convert teacher_id (read as int64) to category
    if 'teacher_id' in merged_df.columns and merged_df['teacher_id'].dtype == 'int64':
         print("Converting 'teacher_id' (int64) to category...")
         merged_df['teacher_id'] = merged_df['teacher_id'].astype('category')

    # Columns to convert to 'datetime' (were object, parse_dates might have failed)
    cols_to_datetime = [
        'timestamp', # from actions_df
        'release_date', # from performance_df
        'due_date', # from performance_df
        'assignment_start_time', # RENAMED from start_time_perf
        'assignment_end_time', # RENAMED from end_time_perf
        'class_creation_date', # from metrics_df
        'teacher_account_creation_date' # from metrics_df
    ]
    # Filter list to only include columns actually present and are object type
    cols_to_datetime = [col for col in cols_to_datetime if col in merged_df.columns and merged_df[col].dtype == 'object']

    if cols_to_datetime:
        print(f"\nConverting {len(cols_to_datetime)} object columns to datetime (standardizing to UTC)...")
        for col in cols_to_datetime:
            print(f"  Converting {col}...")
            try:
                # Add utc=True to standardize timezones
                merged_df[col] = pd.to_datetime(merged_df[col], errors='coerce', utc=True)
            except Exception as e:
                # Added specific error catching
                print(f"  Error converting {col} to datetime with UTC: {e}")
        print("Datetime conversion attempted (standardized to UTC).")
    else:
        print("\nNo object columns identified for datetime conversion in the list.")

    # Convert float64 columns to float32 if appropriate
    float64_cols = merged_df.select_dtypes(include='float64').columns
    if not float64_cols.empty:
        print(f"\nConverting {len(float64_cols)} float64 columns to float32...")
        for col in float64_cols:
             # Check if column still exists (might be empty problems_df col)
             if col in merged_df.columns:
                 print(f"  Converting {col}...")
                 merged_df[col] = merged_df[col].astype('float32') # float32 can handle NaN
    else:
        print("\nNo float64 columns found for conversion.")

    # Display info after renaming and type correction
    print("\n--- DataFrame Info After Step 2 ---")
    merged_df.info(show_counts=True, memory_usage='deep')
    # Display unique types remaining
    print("\nUnique Dtypes after Step 2:", merged_df.dtypes.unique())

else:
    print("Skipping Step 2: merged_df not available.")

# Clean up memory
gc.collect()


--- Step 2: Renaming Columns & Correcting Dtypes ---
Applying renames: {'start_time_perf': 'assignment_start_time', 'end_time_perf': 'assignment_end_time', 'assistments_reference_action_log_id': 'action_log_id', 'assistments_reference_assignment_log_id': 'assignment_log_id'}

Converting 13 object columns to category...
Category conversion attempted.
Converting 'teacher_id' (int64) to category...

Converting 7 object columns to datetime (standardizing to UTC)...
  Converting timestamp...
  Converting release_date...
  Converting due_date...
  Converting assignment_start_time...
  Converting assignment_end_time...
  Converting class_creation_date...
  Converting teacher_account_creation_date...
Datetime conversion attempted (standardized to UTC).

Converting 7 float64 columns to float32...
  Converting session_count...
  Converting attempt_count...
  Converting hints_available...
  Converting hints_given...
  Converting scaffold_problems_available...
  Converting scaffold_problems_given

0

In [8]:
# --- Step 3: Analyze Missing Values ---

if 'merged_df' in locals() and merged_df is not None:
    print("\n--- Step 3: Missing Value Analysis ---")
    missing_counts = merged_df.isnull().sum()
    missing_percent = (merged_df.isnull().sum() / len(merged_df)) * 100

    # Combine counts and percentages into a DataFrame for easier viewing
    missing_info = pd.DataFrame({
        'Missing Count': missing_counts,
        'Missing Percent': missing_percent
    })

    # Display columns with any missing values, sorted by percentage descending
    print("Columns with Missing Values (Sorted):")
    # Increase display rows to see more columns
    with pd.option_context('display.max_rows', 100):
        print(missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing Percent', ascending=False))

    # --- Note on Empty Problem Columns ---
    # Expect to see columns like 'problem_condition', 'start_time', 'end_time', 'correct', etc.
    # listed with 100% missing values here.

else:
    print("Skipping Step 3: merged_df not available.")


--- Step 3: Missing Value Analysis ---
Columns with Missing Values (Sorted):
                                                  Missing Count  \
scaffold_problems_given                                 3711215   
first_response_or_request_time                          3711215   
hints_given                                             3711215   
hints_available                                         3711215   
attempt_count                                           3711215   
answer_before_tutoring                                  3711215   
reported_score                                          3711215   
correct                                                 3711215   
first_answer                                            3711215   
time_on_task                                            3711215   
explanation_available                                   3711215   
session_count                                           3711215   
end_time                                           

In [9]:
# --- Step 4: Handle Specific Missing Values & Final Type Corrections ---

if 'merged_df' in locals() and merged_df is not None:
    print("\n--- Step 4: Specific Cleaning & Final Corrections ---")

    # 4a. Retry Date Conversion with utc=True for remaining object columns
    # Columns identified as object after Step 2 that should be dates
    cols_to_datetime_retry = [
        'timestamp', 'release_date', 'assignment_start_time',
        'assignment_end_time', 'class_creation_date', 'teacher_account_creation_date'
    ]
    # Filter list to only include columns actually present and are object type
    cols_to_datetime_retry = [col for col in cols_to_datetime_retry if col in merged_df.columns and merged_df[col].dtype == 'object']

    if cols_to_datetime_retry:
        print(f"\nRetrying conversion of {len(cols_to_datetime_retry)} object columns to datetime (UTC)...")
        for col in cols_to_datetime_retry:
            print(f"  Converting {col}...")
            # errors='coerce' turns failures into NaT
            # utc=True standardizes to UTC timezone
            merged_df[col] = pd.to_datetime(merged_df[col], errors='coerce', utc=True)
        print("Datetime conversion retry attempted.")
        # Verify conversion (optional)
        print("Dtypes after datetime retry:", merged_df[cols_to_datetime_retry].dtypes)
    else:
        print("\nNo object columns identified for datetime conversion retry.")


    # 4b. Address Completely Empty Problem Columns (Recommend Dropping)
    # These columns showed 100% missing in Step 3.
    empty_problem_cols_to_drop = [
         'problem_condition', 'start_time', 'end_time', 'session_count', 'time_on_task',
         'first_response_or_request_time', 'first_answer', 'correct', 'reported_score',
         'answer_before_tutoring', 'attempt_count', 'hints_available', 'hints_given',
         'scaffold_problems_available', 'scaffold_problems_given', 'explanation_available',
         'explanation_given', 'answer_given',
         'assistments_reference_problem_log_id'
    ]
    # Filter list to only include columns actually present
    empty_problem_cols_to_drop = [col for col in empty_problem_cols_to_drop if col in merged_df.columns]

    if empty_problem_cols_to_drop:
       print(f"\nDropping {len(empty_problem_cols_to_drop)} empty columns from problems_df merge...")
       # Check if they are indeed fully null before dropping (sanity check)
       fully_null = merged_df[empty_problem_cols_to_drop].isnull().all()
       cols_confirmed_empty = fully_null[fully_null].index.tolist()
       if cols_confirmed_empty:
           print("Confirmed empty, dropping:", cols_confirmed_empty)
           merged_df.drop(columns=cols_confirmed_empty, inplace=True)
       else:
           print("Warning: Columns previously thought empty were not fully empty, not dropping automatically.")
           print("  Columns not dropped:", list(set(empty_problem_cols_to_drop) - set(cols_confirmed_empty)))
    else:
       print("\nNo empty problem columns found/listed to drop.")


    # 4c. Convert 'opportunity_zone' (category) to boolean
    if 'opportunity_zone' in merged_df.columns and isinstance(merged_df['opportunity_zone'].dtype, pd.CategoricalDtype):
        print("\nConverting 'opportunity_zone' to boolean (Yes=True, No=False, others=NA)...")
        bool_map = {'Yes': True, 'No': False}
        # Apply map using .cat.codes or map directly if categories are simple
        # Using map is safer if categories might include non-string types later
        merged_df['opportunity_zone_bool'] = merged_df['opportunity_zone'].map(bool_map)
        merged_df['opportunity_zone_bool'] = merged_df['opportunity_zone_bool'].astype('boolean') # Use pandas nullable boolean
        print("Created 'opportunity_zone_bool' column.")
        print("Value counts (including NA):")
        print(merged_df['opportunity_zone_bool'].value_counts(dropna=False))
        # Consider dropping original: merged_df.drop(columns=['opportunity_zone'], inplace=True)
    elif 'opportunity_zone' in merged_df.columns:
         print("\n'opportunity_zone' column found but is not category type. Skipping boolean conversion.")
    else:
        print("\n'opportunity_zone' column not found.")


    # 4d. Handle other missing values based on Step 3 analysis (Examples)

    # Fill missing district/location/locale with 'Unknown' category
    cat_cols_to_fill_unknown = ['district_id', 'location', 'locale_description']
    # Use corrected type checking
    cat_cols_to_fill_unknown = [col for col in cat_cols_to_fill_unknown if col in merged_df.columns and isinstance(merged_df[col].dtype, pd.CategoricalDtype)]

    if cat_cols_to_fill_unknown:
        print(f"\nFilling NA with 'Unknown' for category columns: {cat_cols_to_fill_unknown}")
        for col in cat_cols_to_fill_unknown:
             # Check if 'Unknown' is already a category, add if not
             if 'Unknown' not in merged_df[col].cat.categories:
                  # Use .cat.add_categories() which is the standard way
                  merged_df[col] = merged_df[col].cat.add_categories(['Unknown'])

             merged_df[col] = merged_df[col].fillna('Unknown')

        print("NA filling for specified categories complete.")
    else:
         print("\nNo specified category columns found/available to fill with 'Unknown'.")

    # Re-check info after changes
    print("\n--- DataFrame Info After Step 4 ---")
    merged_df.info(show_counts=True, memory_usage='deep')

else:
    print("Skipping Step 4: merged_df not available.")

# Clean up memory
gc.collect()


--- Step 4: Specific Cleaning & Final Corrections ---

No object columns identified for datetime conversion retry.

Dropping 19 empty columns from problems_df merge...
Confirmed empty, dropping: ['problem_condition', 'start_time', 'end_time', 'session_count', 'time_on_task', 'first_response_or_request_time', 'first_answer', 'correct', 'reported_score', 'answer_before_tutoring', 'attempt_count', 'hints_available', 'hints_given', 'scaffold_problems_available', 'scaffold_problems_given', 'explanation_available', 'explanation_given', 'answer_given', 'assistments_reference_problem_log_id']

Converting 'opportunity_zone' to boolean (Yes=True, No=False, others=NA)...
Created 'opportunity_zone_bool' column.
Value counts (including NA):
opportunity_zone_bool
True     1567792
False    1252490
<NA>      890933
Name: count, dtype: Int64

Filling NA with 'Unknown' for category columns: ['district_id', 'location', 'locale_description']
NA filling for specified categories complete.

--- DataFrame In

0

In [10]:
# --- Step 5: Final Review & Potential Column Dropping ---

if 'merged_df' in locals() and merged_df is not None:
    print("\n--- Step 5: Final Review & Dropping Unnecessary Columns ---")

    # Review current columns if needed
    print("Current columns:", merged_df.columns.tolist())
    print(f"Current shape: {merged_df.shape}")
    # merged_df.info(verbose=True, show_counts=True, memory_usage='deep')
    # print(merged_df.head())

    # --- Decide which columns are unnecessary for your final analysis ---

    final_cols_to_drop = [
        'opportunity_zone',
    ]
    # Filter list to only include columns actually present in the DataFrame
    final_cols_to_drop = [col for col in final_cols_to_drop if col in merged_df.columns]

    if final_cols_to_drop:
        print("\nDropping final unnecessary columns:", final_cols_to_drop)
        merged_df.drop(columns=final_cols_to_drop, inplace=True)
        print("\nColumns remaining:", merged_df.columns.tolist())
    else:
        print("\nNo final columns specified for dropping.")

    print(f"\nFinal Shape after Cleaning: {merged_df.shape}")
    print("\n--- Cleaning Complete ---")
    print("Final DataFrame Info:")
    merged_df.info(show_counts=True, memory_usage='deep')


    # Clean up memory
    gc.collect()

else:
    print("Skipping Step 5: merged_df not available.")


--- Step 5: Final Review & Dropping Unnecessary Columns ---
Current columns: ['experiment_id', 'student_id', 'problem_id', 'problem_part', 'scaffold_id', 'experiment_tag_path', 'action', 'timestamp', 'action_log_id', 'release_date', 'due_date', 'assignment_start_time', 'assignment_end_time', 'assignment_session_count', 'pretest_problem_count', 'pretest_correct', 'pretest_time_on_task', 'pretest_average_first_response_time', 'pretest_session_count', 'assigned_condition', 'condition_time_on_task', 'condition_average_first_response_or_request_time', 'condition_problem_count', 'condition_total_correct', 'condition_total_correct_after_wrong_response', 'condition_total_correct_after_tutoring', 'condition_total_answers_before_tutoring', 'condition_total_attempt_count', 'condition_total_hints_available', 'condition_total_hints_given', 'condition_total_scaffold_problems_available', 'condition_total_scaffold_problems_given', 'condition_total_explanations_available', 'condition_total_explanation

In [11]:
# --- Step 6: Create Reduced DataFrame Before Aggregation ---

if 'merged_df' in locals() and merged_df is not None:
    print("--- Reducing DataFrame to Essential Columns for Aggregation ---")

    essential_cols_to_keep = [
        # Keys
        'experiment_id',
        'student_id',
        # Time / Action Info
        'timestamp',
        'action',
        'action_log_id',
        # Aggregated ('first') Features Needed Directly or for Derivation
        'assignment_session_count',
        'condition_time_on_task',
        'condition_average_first_response_or_request_time',
        'condition_problem_count',
        'condition_total_correct',
        'condition_total_attempt_count',
        'condition_total_hints_given',
        'condition_total_explanations_given',
        'student_prior_average_correctness',
        'opportunity_zone_bool', # Keep the boolean version
    ]

    # Ensure all essential columns exist in the current merged_df
    missing_essentials = [col for col in essential_cols_to_keep if col not in merged_df.columns]
    if missing_essentials:
        print(f"Error: The following essential columns are missing from merged_df: {missing_essentials}")
        merged_df_reduced = None
    else:
        print(f"Selecting {len(essential_cols_to_keep)} essential columns...")
        # Create a new DataFrame with only these columns
        merged_df_reduced = merged_df[essential_cols_to_keep].copy()

        print("\nReduced DataFrame Info:")
        merged_df_reduced.info(memory_usage='deep')

        # Delete the large merged_df to save memory if no longer needed
        print("\nDeleting original large merged_df...")
        del merged_df
        gc.collect()
        print("Original merged_df deleted.")

else:
    print("Skipping reduction: Cleaned merged_df not found.")
    merged_df_reduced = None

--- Reducing DataFrame to Essential Columns for Aggregation ---
Selecting 15 essential columns...

Reduced DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711215 entries, 0 to 3711214
Data columns (total 15 columns):
 #   Column                                            Dtype              
---  ------                                            -----              
 0   experiment_id                                     category           
 1   student_id                                        category           
 2   timestamp                                         datetime64[ns, UTC]
 3   action                                            category           
 4   action_log_id                                     uint64             
 5   assignment_session_count                          UInt16             
 6   condition_time_on_task                            float32            
 7   condition_average_first_response_or_request_time  float32            
 8   condition

In [12]:
# --- Step 7: Save the cleaned DataFrame ---

# Define the specific, full path where you want to save the file
SAVE_CLEANED_PATH = '/Users/john/Downloads/osfstorage-archive/merged_experiment_data_cleaned.csv'

# Check if the merged DataFrame exists and is not empty before saving
if 'merged_df_reduced' in locals() and merged_df_reduced is not None and not merged_df_reduced.empty:
    print(f"\nAttempting to save cleaned DataFrame to: {SAVE_CLEANED_PATH}")
    try:
        # Ensure the directory exists
        save_dir = os.path.dirname(SAVE_CLEANED_PATH)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir) # Create directory if it doesn't exist

        # Save the DataFrame
        merged_df_reduced.to_csv(SAVE_CLEANED_PATH, index=False, encoding='utf-8-sig')
        print(f"Successfully saved cleaned DataFrame to {SAVE_CLEANED_PATH}")
    except Exception as e:
        print(f"Error saving cleaned DataFrame: {e}")
elif 'merged_df_reduced' in locals() and merged_df_reduced is not None and merged_df_reduced.empty:
     print("\nSkipping save: Merged DataFrame is empty.")
else:
    print("\nSkipping save: Merged DataFrame does not exist or cleaning failed.")


Attempting to save cleaned DataFrame to: /Users/john/Downloads/osfstorage-archive/merged_experiment_data_cleaned.csv
Successfully saved cleaned DataFrame to /Users/john/Downloads/osfstorage-archive/merged_experiment_data_cleaned.csv


# Load Cleaned DataFrame

In [13]:
# --- Configuration ---
file_path_2 = '/Users/john/Downloads/osfstorage-archive/merged_experiment_data_cleaned.csv'

def load_dataframe(file_path_2: str) -> pd.DataFrame:
    """Loads a CSV file into a pandas DataFrame."""
    try:
        df_local = pd.read_csv(file_path_2)
        print(f"Successfully loaded: {file_path_2}")
        return df_local
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path_2}")
        print("Please double-check the file path and ensure your Drive is mounted correctly.")
        return None # Return None or raise the error if you prefer
    except Exception as e:
        print(f"An unexpected error occurred while loading {file_path_2}: {e}")
        return None

# --- Script Execution ---

# Call the function and assign the returned DataFrame to the variable 'df'
df = load_dataframe(file_path_2)

if df is not None:
    # Display the first 5 rows of the DataFrame as an example
    print("\nDataFrame Head:")
    display(df.head()) # Using display here

else:
    print("\nDataFrame could not be loaded due to errors.")

Successfully loaded: /Users/john/Downloads/osfstorage-archive/merged_experiment_data_cleaned.csv

DataFrame Head:


,experiment_id,student_id,timestamp,action,action_log_id,assignment_session_count,condition_time_on_task,condition_average_first_response_or_request_time,condition_problem_count,condition_total_correct,condition_total_attempt_count,condition_total_hints_given,condition_total_explanations_given,student_prior_average_correctness,opportunity_zone_bool
0,PSAU85Y,10408,2021-03-24 16:41:46.393000+00:00,assignment_started,669417322,3,595.56,263.5675,3.0,1.0,5.0,1.0,0.0,0.738739,True
1,PSAU85Y,10408,2021-03-24 16:41:47.437000+00:00,problem_started,669417325,3,595.56,263.5675,3.0,1.0,5.0,1.0,0.0,0.738739,True
2,PSAU85Y,10408,2021-03-24 16:42:03.665000+00:00,correct_response,669418287,3,595.56,263.5675,3.0,1.0,5.0,1.0,0.0,0.738739,True
3,PSAU85Y,10408,2021-03-24 16:42:03.675000+00:00,problem_finished,669418492,3,595.56,263.5675,3.0,1.0,5.0,1.0,0.0,0.738739,True
4,PSAU85Y,10408,2021-03-24 16:42:05.295000+00:00,continue_selected,669418493,3,595.56,263.5675,3.0,1.0,5.0,1.0,0.0,0.738739,True
